
# 🗞️ Daily Report Template

This notebook generates a **Bloomberg-style daily report** with:
- Market snapshot (indices, equities, crypto)
- Top headlines (RSS)
- Weather (Open-Meteo)
- Optional portfolio PnL (if `data/positions.csv` exists)
- Optional quick Risk (VaR/ES) from `data/returns.csv`

> Works online or offline. If web access or certain libraries are unavailable, it falls back to local files or synthetic examples.


## 0. Parameters

In [ ]:

# ==== Configure your report here ====
REPORT_TITLE = "Morning Tape"
TIMEZONE = "Asia/Kolkata"           # for timestamps
DATE_OVERRIDE = None                # e.g. "2025-09-03" or None for today

TICKERS = ["^GSPC", "NDX", "SPY", "AAPL", "MSFT", "TSLA"]
CRYPTO  = ["BTC-USD", "ETH-USD"]

# Some RSS feeds (add your favorites)
RSS_FEEDS = [
    "https://feeds.a.dj.com/rss/RSSMarketsMain.xml",
    "https://www.ft.com/?format=rss"
]

CITY = "Mumbai"
COUNTRY = "India"

TOP_N_HEADLINES = 5

# Output paths
OUT_DIR = "reports"
FILENAME_MD = None     # None -> auto (daily_YYYY-MM-DD.md)
FILENAME_PDF = None    # None -> auto (daily_YYYY-MM-DD.pdf) if weasyprint is available


## 1. Setup & Helpers

In [ ]:

import datetime
import os, sys, math, json, time, pathlib, textwrap
from typing import List, Dict, Any, Optional, Tuple
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Optional deps
try:
    import yfinance as yf
    HAS_YF = True
except Exception as e:
    HAS_YF = False

try:
    import feedparser
    HAS_FEED = True
except Exception as e:
    HAS_FEED = False

try:
    import requests
    HAS_REQ = True
except Exception as e:
    HAS_REQ = False

def today_str(tz_name: Optional[str] = None) -> str:
    # naive date; if tz-aware is needed, can use pytz/dateutil. Keep simple here.
    return (DATE_OVERRIDE or datetime.now().date().isoformat()) # type: ignore

def ensure_dir(d):
    os.makedirs(d, exist_ok=True)


## 2. Market Snapshot

In [ ]:

def fetch_quotes(tickers: List[str]) -> pd.DataFrame:
    if not tickers:
        return pd.DataFrame(columns=["ticker","price","prev_close","chg","chg_pct"])
    df = []
    if HAS_YF:
        try:
            for t in tickers:
                t = t.strip()
                tk = yf.Ticker(t)
                info = getattr(tk, "fast_info", {}) or {}
                price = info.get("last_price")
                prev = info.get("previous_close")
                if price is None:
                    hist = tk.history(period="2d")
                    if not hist.empty:
                        price = float(hist["Close"].iloc[-1])
                        prev = float(hist["Close"].iloc[-2]) if len(hist) > 1 else price
                if (price is None) or (prev is None):
                    continue
                chg = price - prev
                chg_pct = (chg / prev) * 100.0 if prev else None
                df.append({"ticker": t, "price": price, "prev_close": prev, "chg": chg, "chg_pct": chg_pct})
            return pd.DataFrame(df)
        except Exception as e:
            pass
    # Fallback synthetic
    rng = np.random.default_rng(0)
    for t in tickers:
        prev = 100 + rng.normal(0, 1)
        chg_pct = rng.normal(0, 1)
        price = prev * (1 + chg_pct/100.0)
        df.append({"ticker": t, "price": price, "prev_close": prev, "chg": price-prev, "chg_pct": chg_pct})
    return pd.DataFrame(df)

eq_df = fetch_quotes(TICKERS)
cr_df = fetch_quotes(CRYPTO)

display(eq_df)
display(cr_df)


### Plot % Change — Equities/Indices

In [ ]:

if not eq_df.empty:
    plt.figure()
    plt.bar(eq_df["ticker"], eq_df["chg_pct"])
    plt.title("Market Snapshot: Δ% vs Prior Close")
    plt.xlabel("Ticker"); plt.ylabel("Δ%")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


### Plot % Change — Crypto

In [ ]:

if not cr_df.empty:
    plt.figure()
    plt.bar(cr_df["ticker"], cr_df["chg_pct"])
    plt.title("Crypto Snapshot: Δ% vs Prior Close")
    plt.xlabel("Ticker"); plt.ylabel("Δ%")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


## 3. Headlines

In [ ]:

def fetch_rss_items(url: str, top_n: int):
    if HAS_FEED:
        try:
            feed = feedparser.parse(url)
            out = []
            for e in feed.entries[:top_n]:
                title = getattr(e, "title", "").strip()
                link = getattr(e, "link", "").strip()
                pub = getattr(e, "published", getattr(e, "updated", "")) or ""
                out.append({"title": title, "link": link, "published": pub})
            return out
        except Exception:
            pass
    # Fallback sample
    return [{"title": f"Sample headline from {url}", "link": url, "published": today_str()}]

all_news = {}
for u in RSS_FEEDS:
    items = fetch_rss_items(u, TOP_N_HEADLINES)
    all_news[u] = items

for src, items in all_news.items():
    print(f"Source: {src}")
    for it in items:
        print("-", it["title"])
    print()


## 4. Weather

In [ ]:

OPEN_METEO_GEOCODE = "https://geocoding-api.open-meteo.com/v1/search"
OPEN_METEO_FORECAST = "https://api.open-meteo.com/v1/forecast"

def geocode(city: str, country: Optional[str] = None):
    if not HAS_REQ:
        return None
    try:
        params = {"name": city, "count": 1, "language": "en", "format": "json"}
        if country:
            params["country"] = country
        r = requests.get(OPEN_METEO_GEOCODE, params=params, timeout=10)
        if r.status_code != 200:
            return None
        data = r.json()
        if not data.get("results"):
            return None
        res = data["results"][0]
        return float(res["latitude"]), float(res["longitude"]), res.get("timezone", "UTC")
    except Exception:
        return None

def forecast(lat: float, lon: float, tz_name: str):
    if not HAS_REQ:
        return None
    try:
        params = {
            "latitude": lat, "longitude": lon, "timezone": tz_name,
            "daily": "weathercode,temperature_2m_max,temperature_2m_min,precipitation_sum,wind_speed_10m_max",
            "forecast_days": 3
        }
        r = requests.get(OPEN_METEO_FORECAST, params=params, timeout=10)
        if r.status_code != 200:
            return None
        return r.json()
    except Exception:
        return None

weather = None
geo = geocode(CITY, COUNTRY)
if geo:
    lat, lon, tz_name = geo
    weather = forecast(lat, lon, tz_name)

if weather:
    daily = weather.get("daily", {})
    dates = daily.get("time", [])[:3]
    tmax = daily.get("temperature_2m_max", [])[:3]
    tmin = daily.get("temperature_2m_min", [])[:3]
    rain = daily.get("precipitation_sum", [])[:3]
    wind = daily.get("wind_speed_10m_max", [])[:3]
    dfw = pd.DataFrame({"date": dates, "low_C": tmin, "high_C": tmax, "rain_mm": rain, "wind_mps": wind})
else:
    # Fallback sample
    dfw = pd.DataFrame({
        "date": [today_str(),],
        "low_C": [24.0],
        "high_C": [31.0],
        "rain_mm": [5.0],
        "wind_mps": [4.0]
    })

display(dfw)


## 5. Portfolio PnL (optional)

In [ ]:

def load_positions(path="data/positions.csv"):
    if os.path.exists(path):
        df = pd.read_csv(path)
        # Expected columns: ticker, quantity, avg_price
        df["ticker"] = df["ticker"].astype(str)
        return df
    return pd.DataFrame(columns=["ticker","quantity","avg_price"])

pos = load_positions()
pnl_df = pd.DataFrame(columns=["ticker","quantity","avg_price","mkt_price","unrealized_pnl"])

if not pos.empty and not eq_df.empty:
    prices = dict(zip(eq_df["ticker"], eq_df["price"]))
    # include crypto too
    prices.update(dict(zip(cr_df["ticker"], cr_df["price"])))
    rows = []
    for _, r in pos.iterrows():
        px = prices.get(r["ticker"])
        if px is None:
            continue
        pnl = (px - float(r["avg_price"])) * float(r["quantity"])
        rows.append({"ticker": r["ticker"], "quantity": r["quantity"], "avg_price": r["avg_price"], "mkt_price": px, "unrealized_pnl": pnl})
    pnl_df = pd.DataFrame(rows)

display(pnl_df)
if not pnl_df.empty:
    print("Total Unrealized PnL:", float(pnl_df["unrealized_pnl"].sum()))


## 6. Quick Risk (VaR / ES) — optional

In [ ]:

def quick_var_es(returns: pd.Series, alpha=0.95) -> Dict[str, float]:
    r = returns.dropna().sort_values()
    if r.empty:
        return {"var": np.nan, "es": np.nan}
    # Historical VaR/ES
    idx = int((1-alpha) * len(r))
    idx = max(0, min(idx, len(r)-1))
    var = -r.iloc[idx]
    es = -r.iloc[:idx+1].mean() if idx >= 0 else np.nan
    return {"var": float(var), "es": float(es)}

risk_summary = {}
if os.path.exists("data/returns.csv"):
    rets = pd.read_csv("data/returns.csv", parse_dates=["date"]).set_index("date").sort_index()
    # Equal-weight portfolio return
    port = rets.mean(axis=1)
    risk_summary = quick_var_es(port.pct_change().dropna() if port.abs().max()>1 else port.dropna(), alpha=0.95)

risk_summary


## 7. Compose & Save Report (Markdown)

In [ ]:

def fmt_quotes(df: pd.DataFrame) -> str:
    if df is None or df.empty:
        return "_unavailable_\n"
    lines = ["| Ticker | Price | Δ | Δ% |", "|---|---:|---:|---:|"]
    for _, r in df.iterrows():
        p = f"{r['price']:.2f}"
        d = f"{r['chg']:+.2f}"
        dp = f"{r['chg_pct']:+.2f}%"
        lines.append(f"| {r['ticker']} | {p} | {d} | {dp} |")
    return "\n".join(lines) + "\n"

def fmt_weather(dfw: pd.DataFrame) -> str:
    lines = ["| Date | Low (°C) | High (°C) | Rain (mm) | Wind (m/s) |", "|---|---:|---:|---:|---:|"]
    for _, r in dfw.iterrows():
        lines.append(f"| {r['date']} | {r['low_C']:.1f} | {r['high_C']:.1f} | {r['rain_mm']:.1f} | {r['wind_mps']:.1f} |")
    return "\n".join(lines) + "\n"

def fmt_headlines(news: Dict[str, list]) -> str:
    out = []
    for src, items in news.items():
        out.append(f"**{src}**")
        if not items:
            out.append("- _(no items)_")
            continue
        for it in items:
            title = it['title'] or '(untitled)'
            link = it['link']
            pub = it.get('published','')
            out.append(f"- {title} — {pub}\n  <{link}>")
        out.append("")
    return "\n".join(out)

datestamp = today_str()
ensure_dir(OUT_DIR)
md_name = FILENAME_MD or f"daily_{datestamp}.md"
md_path = os.path.join(OUT_DIR, md_name)

parts = []
parts.append(f"# {REPORT_TITLE} — {datestamp}\n")
parts.append(f"_Generated {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}_\n") # type: ignore
parts.append("## Market Snapshot\n" + fmt_quotes(eq_df))
if not cr_df.empty:
    parts.append("## Crypto Snapshot\n" + fmt_quotes(cr_df))
parts.append("## Weather\n" + fmt_weather(dfw))
parts.append("## Top Headlines\n" + fmt_headlines(all_news))

if not pnl_df.empty:
    lines = ["| Ticker | Qty | Avg Px | Mkt Px | Unrealized PnL |", "|---|---:|---:|---:|---:|"]
    for _, r in pnl_df.iterrows():
        lines.append(f"| {r['ticker']} | {r['quantity']} | {float(r['avg_price']):.2f} | {float(r['mkt_price']):.2f} | {float(r['unrealized_pnl']):+.2f} |")
    parts.append("## Portfolio PnL\n" + "\n".join(lines) + "\n")
    parts.append(f"**Total Unrealized PnL:** {float(pnl_df['unrealized_pnl'].sum()):+.2f}\n")

if risk_summary:
    parts.append(f"## Risk (95% VaR/ES)\n- VaR: {risk_summary.get('var', float('nan')):.4f}\n- ES: {risk_summary.get('es', float('nan')):.4f}\n")

md = "\n\n".join(parts)
with open(md_path, "w", encoding="utf-8") as f:
    f.write(md + "\n")

print("Wrote:", md_path)
md[:500] + ("...\n" if len(md)>500 else "")


## 8. Optional — Export PDF (WeasyPrint, if installed)

In [ ]:

PDF_WRITTEN = None
try:
    from weasyprint import HTML, CSS # type: ignore
    # Minimal Markdown -> HTML (very light)
    def md_to_html_simple(s: str) -> str:
        # very naive conversion for headers and lists/tables
        import re
        s = s.replace("\n\n", "<br/><br/>")
        s = re.sub(r"^# (.*)$", r"<h1>\1</h1>", s, flags=re.MULTILINE)
        s = re.sub(r"^## (.*)$", r"<h2>\1</h2>", s, flags=re.MULTILINE)
        return f"<!doctype html><html><body>{s}</body></html>"
    html = md_to_html_simple(md)
    pdf_name = FILENAME_PDF or f"daily_{datestamp}.pdf"
    pdf_path = os.path.join(OUT_DIR, pdf_name)
    HTML(string=html).write_pdf(pdf_path)
    PDF_WRITTEN = pdf_path
    print("Wrote PDF:", pdf_path)
except Exception as e:
    print("PDF export skipped (WeasyPrint not available or failed).")
